In [1]:
from dotenv import load_dotenv
load_dotenv()

from os import getenv
from json import dump
from collections import defaultdict
import sqlite3
import numpy as np
import torch
from scipy.sparse import csr_matrix
from transformers import AutoTokenizer, AutoModelForMaskedLM

/Users/Matic/Documents/School/Magistrska/Monorepo/components/notebooks/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DB_PATH = "./output/sparse_index.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

In [3]:
MODEL_NAME = "naver/splade-cocondenser-ensembledistil"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)
model.eval()

model.device

device(type='mps', index=0)

In [4]:
def search_index(query, k):
	tokens = tokenizer(query, return_tensors='pt', padding=False, truncation=False)
	if tokens['input_ids'].shape[1] > 512:
		raise ValueError("Input text is too long")

	tokens = {k: v.to(model.device) for k, v in tokens.items()}

	with torch.no_grad():
		outputs = model(**tokens)

	vector = torch.max(
		torch.log(1 + torch.relu(outputs.logits)) * tokens['attention_mask'].unsqueeze(-1),
		dim=1
	)[0].squeeze()

	indices = vector.nonzero().squeeze().cpu().tolist()
	if not isinstance(indices, list):
		indices = [indices]

	if len(indices) == 0:
		return []

	weights = vector[indices].cpu().tolist()

	params = []
	for idx, score in zip(indices, weights):
		params.extend([int(idx), float(score)])
	params.append(k)

	values_placeholders = ', '.join(['(?,?)'] * len(indices))

	sql_query = f'''
		WITH query_terms(term, score) AS (
			VALUES {values_placeholders}
		)
		SELECT
			d.filename AS document,
			SUM(idx.score * q.score) AS total_score
		FROM
			inverted_index AS idx
		JOIN
			query_terms AS q ON idx.term = q.term
		JOIN
			documents AS d ON idx.document_id = d.id
		GROUP BY
			idx.document_id, d.filename
		ORDER BY
			total_score DESC
		LIMIT ?
	'''

	cursor.execute(sql_query, params)
	return cursor.fetchall()

In [5]:
results = search_index("state of the art image segmentation", k=5)
for filename, distance in results:
    print(f"{filename}: {distance:.4f}")

LPeoJSBciKrMfVGnis9rvg: 20.8188
a84sFqjaRXr4N_hLI9vCWQ: 20.3556
XFFwdctZiWtBfwkRRL2l3A: 19.0164
NmRj10H0M6CFxiaySnKtkA: 18.7227
Bau9eDjWEeQLfK-5uVzo-Q: 18.6545
